# Parameter Fine Tuning
based on: Udemy DEEP LEARNING A-Z™ Course
### Setup
To setup, we need to have these packages installed: <br>
1- Tensorflow<br>
2- Keras<br>
3- Theano | we will not work with this <br>

### A classification problem
based on the multi-dimensional data

In [1]:
import tensorflow as tf
# Theano is a mathematical library based on numpy, that can run on the GPU
#import theano
import keras

# we will need these for the ANN
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# will need this for data pre-processing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# will need this for splitting the data set
from sklearn.model_selection import train_test_split

# will need it for the scaling!
from sklearn.preprocessing import StandardScaler

# will need to calculate the loss
from sklearn.metrics import confusion_matrix

################################################################################
# will need it for the cross validation
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import cross_val_score

################################################################################
# will need it for the grid search
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


## Data pre-processing
Reading data from the CSV file and make it ready for the next steps.

### Imporet data

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
# we only need features from 3 to 12 (upper bound is not inclusive)
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values
print (X[0])

[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]


### Encoding categorical data
There are two variables that are categorial, country and gender <br>
So, we convert them into one integer values! So if we apply it to country, it will replcae France, Germany, and Spain with 0,1,2! Nice!

In [3]:
# this will handle the country
labelEncoderX01 = LabelEncoder()
X[:,1] = labelEncoderX01.fit_transform(X[:,1])

# this will handle the gender
labelEncoderX02 = LabelEncoder()
X[:,2] = labelEncoderX02.fit_transform(X[:,2])

### Dummy coding
Now, we need to make some dummy coding! This will prevent the data to make this impression that categoprial values are actually impying any order. Spain is not higher or lower than France nor Germany.
We don't ned to do it with gender, since it only has two variables.
It basically make that 1,2,3 for countries into 3 different variables:
France  0, 0, 1
Germany 0, 1, 0
Spain   1, 0, 0
But we need to remove one of them, because we don't need the third one (if it is not German or French, then it is Spanish!)

In [4]:
oneHotEncoder = OneHotEncoder(categorical_features=[1])
X= oneHotEncoder.fit_transform(X).toarray()
X = X[:, 1:]

### Splitting the data into training and test
Easy peasy!

In [5]:
XTrain, XTest, yTrain, yTest = train_test_split(X,y, test_size = 0.2, random_state = 0)

### Feature Scaling
We must normalize the data, otherwise one feature is going to dominate the others! 

In [6]:
sc = StandardScaler()
# we first need to find the scale
XTrain = sc.fit_transform(XTrain)
# then we apply the same scale to the test set!
XTest = sc.transform(XTest)

# Cross Validation / K-Fold
This is a function that makes the ANN architecture like the previous section, but based on the Scikit_learn

In [10]:
def buildClassifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units= 6, kernel_initializer= "glorot_uniform",activation = "relu",input_dim= 11))
    classifier.add(Dense(units= 6, kernel_initializer= "glorot_uniform",activation = "relu"))
    classifier.add(Dense(units= 1, kernel_initializer= "glorot_uniform",activation = "sigmoid"))
    classifier.compile(optimizer = optimizer, loss ='binary_crossentropy',metrics = ['accuracy'])
    return classifier

Do not add n_jobs = -1 if you use GPU, it will use only one CUDA core!

In [11]:
%%time
classifier =  KerasClassifier(build_fn = buildClassifier)

Wall time: 0 ns


In [12]:
%%time
parameters = {'batch_size': [200,4000],
             'epochs': [100,500],
             'optimizer': ['adam','rmsprop']}
gridSearch = GridSearchCV(estimator = classifier,
                          param_grid = parameters,
                          scoring= 'accuracy', 
                          cv=5)
gridSearch = gridSearch.fit(XTrain,yTrain)

Epoch 1/25
6400/6400 [==============================] - 2s 256us/step - loss: 0.6885 - acc: 0.6069
Epoch 2/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.6840 - acc: 0.6427
Epoch 3/25
6400/6400 [==============================] - 0s 5us/step - loss: 0.6792 - acc: 0.6614
Epoch 4/25
6400/6400 [==============================] - 0s 5us/step - loss: 0.6743 - acc: 0.6767
Epoch 5/25
6400/6400 [==============================] - 0s 5us/step - loss: 0.6694 - acc: 0.6909
Epoch 6/25
6400/6400 [==============================] - 0s 5us/step - loss: 0.6645 - acc: 0.7009
Epoch 7/25
6400/6400 [==============================] - 0s 5us/step - loss: 0.6594 - acc: 0.7108
Epoch 8/25
6400/6400 [==============================] - 0s 5us/step - loss: 0.6543 - acc: 0.7206
Epoch 9/25
6400/6400 [==============================] - 0s 9us/step - loss: 0.6493 - acc: 0.7319
Epoch 10/25
6400/6400 [==============================] - 0s 8us/step - loss: 0.6440 - acc: 0.7387
Epoch 11/25
6400/6400 [====

6400/6400 [==============================] - 0s 5us/step - loss: 0.5668 - acc: 0.7947
Epoch 10/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5616 - acc: 0.7978
Epoch 11/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5567 - acc: 0.7983
Epoch 12/25
6400/6400 [==============================] - 0s 5us/step - loss: 0.5521 - acc: 0.7997
Epoch 13/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5477 - acc: 0.8003
Epoch 14/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5436 - acc: 0.8011
Epoch 15/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5397 - acc: 0.8016
Epoch 16/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5358 - acc: 0.8014
Epoch 17/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5322 - acc: 0.8019
Epoch 18/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5287 - acc: 0.8016
Epoch 19/25
6400/6400 [=========

6400/6400 [==============================] - 0s 4us/step - loss: 0.5249 - acc: 0.7963
Epoch 18/25
6400/6400 [==============================] - 0s 4us/step - loss: 0.5207 - acc: 0.7989
Epoch 19/25
6400/6400 [==============================] - 0s 8us/step - loss: 0.5163 - acc: 0.8000
Epoch 20/25
6400/6400 [==============================] - 0s 7us/step - loss: 0.5123 - acc: 0.8002
Epoch 21/25
6400/6400 [==============================] - 0s 14us/step - loss: 0.5085 - acc: 0.7995
Epoch 22/25
6400/6400 [==============================] - 0s 9us/step - loss: 0.5048 - acc: 0.8000
Epoch 23/25
6400/6400 [==============================] - 0s 6us/step - loss: 0.5011 - acc: 0.8003
Epoch 24/25
6400/6400 [==============================] - 0s 9us/step - loss: 0.4975 - acc: 0.8011
Epoch 25/25
6400/6400 [==============================] - 0s 20us/step - loss: 0.4940 - acc: 0.8011
Epoch 1/25
6400/6400 [==============================] - 0s 55us/step - loss: 1.0197 - acc: 0.3977
Epoch 2/25
6400/6400 [========

6400/6400 [==============================] - 0s 4us/step - loss: 0.6132 - acc: 0.7853
Epoch 1/50
6400/6400 [==============================] - 0s 65us/step - loss: 0.7558 - acc: 0.3984
Epoch 2/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.7472 - acc: 0.4147
Epoch 3/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.7384 - acc: 0.4366
Epoch 4/50
6400/6400 [==============================] - 0s 8us/step - loss: 0.7299 - acc: 0.4575
Epoch 5/50
6400/6400 [==============================] - 0s 8us/step - loss: 0.7217 - acc: 0.4783
Epoch 6/50
6400/6400 [==============================] - 0s 19us/step - loss: 0.7139 - acc: 0.4994
Epoch 7/50
6400/6400 [==============================] - 0s 7us/step - loss: 0.7066 - acc: 0.5167
Epoch 8/50
6400/6400 [==============================] - 0s 7us/step - loss: 0.6996 - acc: 0.5405
Epoch 9/50
6400/6400 [==============================] - 0s 16us/step - loss: 0.6931 - acc: 0.5591
Epoch 10/50
6400/6400 [===============

6400/6400 [==============================] - 0s 5us/step - loss: 0.5687 - acc: 0.7653
Epoch 34/50
6400/6400 [==============================] - 0s 4us/step - loss: 0.5650 - acc: 0.7695
Epoch 35/50
6400/6400 [==============================] - 0s 4us/step - loss: 0.5615 - acc: 0.7734
Epoch 36/50
6400/6400 [==============================] - 0s 4us/step - loss: 0.5581 - acc: 0.7766
Epoch 37/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.5548 - acc: 0.7795
Epoch 38/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.5517 - acc: 0.7811
Epoch 39/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.5486 - acc: 0.7842
Epoch 40/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.5457 - acc: 0.7862
Epoch 41/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.5428 - acc: 0.7877
Epoch 42/50
6400/6400 [==============================] - 0s 4us/step - loss: 0.5401 - acc: 0.7889
Epoch 43/50
6400/6400 [=========

...

6400/6400 [==============================] - 0s 3us/step - loss: 0.7034 - acc: 0.5480
Epoch 15/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6979 - acc: 0.5598
Epoch 16/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6927 - acc: 0.5745
Epoch 17/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6876 - acc: 0.5886
Epoch 18/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6826 - acc: 0.5986
Epoch 19/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6777 - acc: 0.6081
Epoch 20/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6730 - acc: 0.6200
Epoch 21/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6685 - acc: 0.6303
Epoch 22/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6641 - acc: 0.6441
Epoch 23/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6597 - acc: 0.6539
Epoch 24/50
6400/6400 [=========

6400/6400 [==============================] - 0s 5us/step - loss: 0.5170 - acc: 0.7925
Epoch 48/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.5154 - acc: 0.7927
Epoch 49/50
6400/6400 [==============================] - 0s 4us/step - loss: 0.5139 - acc: 0.7930
Epoch 50/50
6400/6400 [==============================] - 0s 11us/step - loss: 0.5124 - acc: 0.7934
Epoch 1/50
6400/6400 [==============================] - 1s 155us/step - loss: 0.8003 - acc: 0.5134
Epoch 2/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.7812 - acc: 0.5270
Epoch 3/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.7689 - acc: 0.5359
Epoch 4/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.7588 - acc: 0.5452
Epoch 5/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.7500 - acc: 0.5523
Epoch 6/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.7421 - acc: 0.5589
Epoch 7/50
6400/6400 [=============

6400/6400 [==============================] - 0s 3us/step - loss: 0.6876 - acc: 0.5795
Epoch 31/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6835 - acc: 0.5873
Epoch 32/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6796 - acc: 0.5989
Epoch 33/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6758 - acc: 0.6094
Epoch 34/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.6721 - acc: 0.6216
Epoch 35/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.6686 - acc: 0.6319
Epoch 36/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6652 - acc: 0.6412
Epoch 37/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6618 - acc: 0.6520
Epoch 38/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6586 - acc: 0.6597
Epoch 39/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6555 - acc: 0.6702
Epoch 40/50
6400/6400 [=========

6400/6400 [==============================] - 0s 2us/step - loss: 0.5762 - acc: 0.7728
Epoch 14/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.5734 - acc: 0.7752
Epoch 15/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.5706 - acc: 0.7775
Epoch 16/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.5679 - acc: 0.7805
Epoch 17/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.5653 - acc: 0.7855
Epoch 18/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.5627 - acc: 0.7872
Epoch 19/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.5601 - acc: 0.7908
Epoch 20/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.5575 - acc: 0.7930
Epoch 21/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.5551 - acc: 0.7947
Epoch 22/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.5527 - acc: 0.7961
Epoch 23/50
6400/6400 [=========

6400/6400 [==============================] - 0s 3us/step - loss: 0.6480 - acc: 0.7109
Epoch 47/50
6400/6400 [==============================] - 0s 2us/step - loss: 0.6452 - acc: 0.7173
Epoch 48/50
6400/6400 [==============================] - 0s 0us/step - loss: 0.6424 - acc: 0.7245
Epoch 49/50
6400/6400 [==============================] - 0s 5us/step - loss: 0.6396 - acc: 0.7297
Epoch 50/50
6400/6400 [==============================] - 0s 3us/step - loss: 0.6369 - acc: 0.7344
Epoch 1/50
8000/8000 [==============================] - 1s 136us/step - loss: 0.6999 - acc: 0.5515
Epoch 2/50
8000/8000 [==============================] - 0s 4us/step - loss: 0.6953 - acc: 0.5594
Epoch 3/50
8000/8000 [==============================] - 0s 6us/step - loss: 0.6902 - acc: 0.5698
Epoch 4/50
8000/8000 [==============================] - 0s 2us/step - loss: 0.6850 - acc: 0.5790
Epoch 5/50
8000/8000 [==============================] - 0s 4us/step - loss: 0.6798 - acc: 0.5868
Epoch 6/50
8000/8000 [=============

In [13]:
bestParameters = gridSearch.best_params_
bestAccuracy = gridSearch.best_score_
print(bestParameters)
print(bestAccuracy)

{'batch_size': 4000, 'epochs': 50, 'optimizer': 'adam'}
0.797375


In [10]:
bestParameters = gridSearch.best_params_
bestAccuracy = gridSearch.best_score_

In [11]:
# returns the 10 accuracies from the 10 fold cross validation
#acuracies = cross_val_score(estimator = classifier,X = XTrain, y = yTrain, cv = 10)#, n_jobs = -1) 

In [12]:
print(bestParameters)
print(bestAccuracy)

{'batch_size': 2000, 'nb_epoch': 100, 'optimizer': 'adam'}
0.796


In [32]:
print(gridSearch)

GridSearchCV(cv=5, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002A4E75B2BA8>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'batch_size': [2000, 4000], 'nb_epoch': [100, 500], 'optimizer': ['adam', 'rmsprop']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)
